# Building a chatbot

We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation an remember previous interactions.

This chatbot will be able to only use the language model to have a conversation. There are several other related concepts like
* Conversational RAg: Enable a chatbot experience over an external source of data
* Agents: Build a chatbot tht can take actions

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model ="Gemma2-9b-It",groq_api_key=groq_api_key)

In [4]:
from langchain_core.messages import HumanMessage
model.invoke([
    HumanMessage(content="Hi,My name is Poorna and a programmer")
])

AIMessage(content="Hello Poorna! \n\nIt's nice to meet you.  What kind of programming do you enjoy doing?  \n\nDo you have any projects you're working on right now that you'd like to talk about? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 19, 'total_tokens': 71, 'completion_time': 0.094545455, 'prompt_time': 8.781e-05, 'queue_time': 0.01337743, 'total_time': 0.094633265}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-d3e532db-4e5a-4dfe-b2be-e45991240381-0', usage_metadata={'input_tokens': 19, 'output_tokens': 52, 'total_tokens': 71})

In [6]:
from langchain_core.messages import AIMessage

model.invoke([
    HumanMessage(content="Hi,my name is poorna and a programmer"),
    AIMessage(content = "Hello Poorna! \n\nIt's nice to meet you.  What kind of programming do you enjoy doing?  \n\nDo you have any projects you're working on right now that you'd like to talk about?"),
    HumanMessage(content = "hey what's my name and what do i do?")

])

AIMessage(content="You said your name is Poorna and that you are a programmer!  \n\nIs there anything else you'd like to tell me about yourself? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 88, 'total_tokens': 123, 'completion_time': 0.063636364, 'prompt_time': 0.003687259, 'queue_time': 0.01004893, 'total_time': 0.067323623}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-0d542301-552a-4aad-b28d-2cc99826350d-0', usage_metadata={'input_tokens': 88, 'output_tokens': 35, 'total_tokens': 123})

In LangChain chatbot development, message history is a crucial component that enables the chatbot to maintain context and understand the flow of conversation. It's essentially a record of the past interactions between the user and the chatbot, including both the user's messages and the chatbot's responses.

**How message history is used:**

* **Contextual awareness:** By referring to the message history, the chatbot can understand the ongoing conversation and avoid repetitive or irrelevant responses. It allows the chatbot to "remember" what has been said earlier and use that information to generate more meaningful and coherent replies.
* **Personalized responses:** Message history helps the chatbot to learn about the user's preferences, interests, and communication style over time. This allows the chatbot to tailor its responses to the individual user, making the interaction more personalized and engaging.
* **Multi-turn conversations:** Message history is essential for supporting multi-turn conversations where the user and the chatbot exchange multiple messages back and forth. It enables the chatbot to follow the thread of the conversation and maintain a consistent dialogue.
* **Improved accuracy:** By considering the context provided by the message history, the chatbot can generate more accurate and relevant responses. This is particularly important for complex or ambiguous queries where the user's intent may not be clear from a single message.

**Implementing message history in LangChain:**

LangChain provides several ways to incorporate message history into your chatbot:

* **`RunnableWithMessageHistory`:** This class allows you to add message history to certain types of chains. It wraps another Runnable and manages the chat message history for it.
* **`LangGraph`:** This feature provides a built-in persistence layer, allowing chain states, including message history, to be automatically persisted in memory or external backends like SQLite, Postgres, or Redis.
* **Custom implementations:** You can also implement your own message history management using lists or other data structures to store and access past messages.

**Example:**

```python
from langchain.chat_models import ChatOpenAI
from langchain.chains import RunnableWithMessageHistory
from langchain.schema import HumanMessage

chat_model = ChatOpenAI()
runnable_with_history = RunnableWithMessageHistory(chat_model)

# Initialize the conversation
runnable_with_history.run(HumanMessage(content="Hi!"))

# Continue the conversation
response = runnable_with_history.run(HumanMessage(content="What's my name?"))
print(response)
```

In this example, `RunnableWithMessageHistory` keeps track of the conversation history, allowing the chatbot to "remember" the initial greeting and respond appropriately to the second message.

By effectively utilizing message history, you can create more engaging, personalized, and contextually aware chatbots with LangChain.


In [9]:
## Message History
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import RunnableWithMessageHistory


store={}
def get_session_history(session_id:str)-> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history =  RunnableWithMessageHistory(model, get_session_history)


In [10]:
config = {'configurable':{'session_id':'chat1'}}


In [17]:
response = with_message_history.invoke(
    [
        HumanMessage(content = 'Hi, My name is Poorna and am the biggest fan of YS Jagan Mohan Reddy.'),
        
    ],config = config

)
response

AIMessage(content="Hi Poorna! It's great to meet you.  \n\nIt's fantastic that you're such a big fan of YS Jagan Mohan Reddy. What do you admire most about him and his work?  \n\nI'm always interested in hearing about what motivates people's political passions. 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 67, 'prompt_tokens': 212, 'total_tokens': 279, 'completion_time': 0.121818182, 'prompt_time': 0.008365234, 'queue_time': 0.007126736000000002, 'total_time': 0.130183416}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3a3403bc-6642-4690-8d44-cff935577df8-0', usage_metadata={'input_tokens': 212, 'output_tokens': 67, 'total_tokens': 279})

In [22]:
response.content

"Hi Poorna! It's great to meet you.  \n\nIt's fantastic that you're such a big fan of YS Jagan Mohan Reddy. What do you admire most about him and his work?  \n\nI'm always interested in hearing about what motivates people's political passions. 😊 \n"

In [21]:
with_message_history.invoke(
    [
        HumanMessage(content = " Whom do i admire most?"),
    ],
    config = config
)

AIMessage(content="You've said you are the biggest fan of YS Jagan Mohan Reddy.  \n\nIt seems you admire him the most! 😊  \n\n\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 431, 'total_tokens': 463, 'completion_time': 0.058181818, 'prompt_time': 0.017182446, 'queue_time': 0.002578740999999999, 'total_time': 0.075364264}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3773e560-9d11-446b-a3c7-c04e0def1086-0', usage_metadata={'input_tokens': 431, 'output_tokens': 32, 'total_tokens': 463})

In [25]:
## change the config ---> changing the seession_id
config1 = {'configurable':{'session_id':'chat2'}}

result = with_message_history.invoke(
    [
        HumanMessage(content ="what is my name?")
    ],
    config = config1
)

result.content

"As an AI, I don't have access to any personal information about you, including your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n"